In [ ]:
!pip install datasets transformers daaja fugashi ipadic numpy

In [29]:
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments 

In [30]:
dataset = load_dataset('subsets',
                       data_files={
                           'train': 'aug_5000_train.tsv',
                           'valid': 'valid.tsv',
                           'test': 'test.tsv'
                        },
                       column_names=["label", "sentence"],
                       delimiter="\t")

Using custom data configuration subsets-ac07db0a8f8d01a1


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/subsets-ac07db0a8f8d01a1/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [31]:
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=9)

loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/573af37b6c39d672f2df687c06ad7d556476cbe43e5bf7771097187c45a3e7bf.abeb707b5d79387dd462e8bfb724637d856e98434b6931c769b8716c6f287258
Model config BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32

In [32]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, max_length=128, padding="max_length")

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [33]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    
args=TrainingArguments(
    output_dir='./outputs',
    learning_rate=1e-5,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=256
)

trainer=Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization st

Step,Training Loss
500,0.758400
1000,0.244400
1500,0.107200
2000,0.040000
2500,0.021100
3000,0.007200
3500,0.003500
4000,0.002400
4500,0.001600
5000,0.001300


Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./outputs/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./outputs/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./outputs/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./outputs/checkpoint-1500
Configuration saved in ./outputs/checkpoint-1500/config.json
Model weights saved in ./outputs/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./outputs/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./outputs/checkpoint-1500/special_toke

TrainOutput(global_step=7820, training_loss=0.07615633297263814, metrics={'train_runtime': 5564.9681, 'train_samples_per_second': 44.924, 'train_steps_per_second': 1.405, 'total_flos': 1.6445474496e+16, 'train_loss': 0.07615633297263814, 'epoch': 10.0})

In [34]:
trainer.evaluate(encoded_dataset["test"])

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 1184
  Batch size = 256


{'epoch': 10.0,
 'eval_accuracy': 0.9113175675675675,
 'eval_loss': 0.606023371219635,
 'eval_runtime': 8.5982,
 'eval_samples_per_second': 137.703,
 'eval_steps_per_second': 0.582}